# **Laboratory 2 TDSE**
Juan Sebastian Puentes Julio

# 1. **SETUP**

In [13]:
%pip install numpy pandas matplotlib

In [12]:
import kagglehub
import pandas as pd
import os

# 2. **DATASET**

In [15]:
path = kagglehub.dataset_download("neurocipher/heartdisease")

df = pd.read_csv(os.path.join(path,'Heart_Disease_Prediction.csv'))
df.head()



Using Colab cache for faster access to the 'heartdisease' dataset.


,Age,Sex,Chest pain type,BP,Cholesterol,FBS over 120,EKG results,Max HR,Exercise angina,ST depression,Slope of ST,Number of vessels fluro,Thallium,Heart Disease
0,70,1,4,130,322,0,2,109,0,2.4,2,3,3,Presence
1,67,0,3,115,564,0,2,160,0,1.6,2,0,7,Absence
2,57,1,2,124,261,0,0,141,0,0.3,1,0,7,Presence
3,64,1,4,128,263,0,0,105,1,0.2,2,1,7,Absence
4,74,0,2,120,269,0,2,121,1,0.2,1,1,3,Absence
